<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#include <tbb/tbb.h>
#include <vector>
#include <iostream>
#include <GL/glew.h>
#include <GLFW/glfw3.h>

void generateData(std::vector<int>& data, int width, int height) {
    for (int i = 0; i < width * height; ++i) {
        data[i] = i;
    }
}

void initOpenGL(GLFWwindow*& window) {
    if (!glfwInit()) {
        std::cerr << "Failed to initialize GLFW" << std::endl;
        exit(EXIT_FAILURE);
    }
    window = glfwCreateWindow(800, 600, "OpenGL", NULL, NULL);
    if (!window) {
        std::cerr << "Failed to create GLFW window" << std::endl;
        glfwTerminate();
        exit(EXIT_FAILURE);
    }
    glfwMakeContextCurrent(window);
    if (glewInit() != GLEW_OK) {
        std::cerr << "Failed to initialize GLEW" << std::endl;
        exit(EXIT_FAILURE);
    }
}

int main() {
    // Initialize OpenGL context
    GLFWwindow* window;
    initOpenGL(window);

    // 텍스처 크기 정의 (예시로 10x10)
    const int width = 10;
    const int height = 10;
    const int dataSize = width * height;
    const int numTextures = 2;

    // 병렬 작업에 사용할 데이터 벡터 생성
    std::vector<int> data1(dataSize);
    std::vector<int> data2(dataSize);

    generateData(data1, width, height);
    generateData(data2, width, height);

    // OpenGL 텍스처 및 PBO 생성
    GLuint textures[numTextures];
    GLuint pbos[numTextures];
    glGenTextures(numTextures, textures);
    glGenBuffers(numTextures, pbos);

    //
    for (int i = 0; i < numTextures; ++i) {
        glBindTexture(GL_TEXTURE_2D, textures[i]);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE);

        glBindBuffer(GL_PIXEL_UNPACK_BUFFER, pbos[i]);
        glBufferData(GL_PIXEL_UNPACK_BUFFER, dataSize * sizeof(int), nullptr, GL_STREAM_DRAW);
    }

    // atomic 변수를 사용하여 결과 합산
    std::atomic<int> total_sum{0};

    /
    tbb::parallel_for(tbb::blocked_range<int>(0, dataSize),
        [&data1, &data2, &pbos, &total_sum](const tbb::blocked_range<int>& range) {
            int local_sum = 0;

            glBindBuffer(GL_PIXEL_UNPACK_BUFFER, pbos[0]);
            int* ptr1 = (int*)glMapBuffer(GL_PIXEL_UNPACK_BUFFER, GL_WRITE_ONLY);
            glBindBuffer(GL_PIXEL_UNPACK_BUFFER, pbos[1]);
            int* ptr2 = (int*)glMapBuffer(GL_PIXEL_UNPACK_BUFFER, GL_WRITE_ONLY);

            for (int i = range.begin(); i != range.end(); ++i) {
                int result1 = data1[i] * 2; // 예: 각 값을 두 배로
                int result2 = data2[i] * 3; // 예: 각 값을 세 배로
                ptr1[i] = result1;
                ptr2[i] = result2;
                local_sum += result1 + result2; // 지역 합산
            }

            glUnmapBuffer(GL_PIXEL_UNPACK_BUFFER);
            glBindBuffer(GL_PIXEL_UNPACK_BUFFER, pbos[1]);
            glUnmapBuffer(GL_PIXEL_UNPACK_BUFFER);

            total_sum += local_sum; // 전체 합산에 지역 합산 추가
        }
    );

    // PBO 데이터를 텍스처로 전송
    glBindTexture(GL_TEXTURE_2D, textures[0]);
    glBindBuffer(GL_PIXEL_UNPACK_BUFFER, pbos[0]);
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, width, height, 0, GL_RGBA, GL_UNSIGNED_BYTE, nullptr);

    glBindTexture(GL_TEXTURE_2D, textures[1]);
    glBindBuffer(GL_PIXEL_UNPACK_BUFFER, pbos[1]);
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, width, height, 0, GL_RGBA, GL_UNSIGNED_BYTE, nullptr);


    std::cout << "Total Sum: " << total_sum.load() << std::endl;

    // OpenGL 루프를 유지하기 위한 코드 (여기서는 간단히 창을 유지하고 종료)
    while (!glfwWindowShouldClose(window)) {
        glClear(GL_COLOR_BUFFER_BIT);
        glfwSwapBuffers(window);
        glfwPollEvents();
    }

    glDeleteTextures(numTextures, textures);
    glDeleteBuffers(numTextures, pbos);
    glfwDestroyWindow(window);
    glfwTerminate();

    return 0;
}